***  EXTRACCION DE CARACTETISTICAS EIGEN  ***

Datos del Drive. 
1. Conexión y permisos con el drive personal

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


2. Ubicación de las carpetas a utilizar

In [ ]:
#%cd /content/gdrive/My Drive/0_Desarrollo_Tesis/2.Resultados_Preprocesamiento/
%cd /content/gdrive/My Drive/0_Desarrollo_Tesis/

/content/gdrive/.shortcut-targets-by-id/13aHaot6tbm7SROiS_JzQgfDtjZ_DM4PH/0_Desarrollo_Tesis


3. Listar la lista del contenido de la ruta

In [ ]:
!ls /content/gdrive/My\ Drive/0_Desarrollo_Tesis/5.Resultados_Histogramas/Eigen_Perros*

***FUNCION HISTOGRAMA***


In [ ]:
def histogram(imgGRAY):
    #histograma
    hist = cv.calcHist([imgGRAY],[0],None,[256],[0,256])
    # return the histogram
    return hist.flatten()

In [ ]:
# CODIGO DE PRUEBAS

def funcion_prueba(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo):
  print("path_EXTRACCION"+path_EXTRACCION)
  print("path_REDIMENSIONADO"+path_REDIMENSIONADO)
  print("anexo_archivo"+anexo_archivo)

*** EIGEN ***

In [ ]:
 # Import matplotlib library 
import matplotlib.pyplot as plt 
import numpy.linalg as linalg 
import numpy as np 
import cv2
import os


import matplotlib.pyplot as plt 
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from skimage import io, color
import cv2 as cv


def extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO, anexo_archivo):

  path_redimensionadas=path_REDIMENSIONADO #Imagen redimensionada a color
  path_resultado=path_EXTRACCION  #Imagen Eigen
  os.makedirs(path_resultado, exist_ok=True)

  KA=anexo_archivo.split(sep='/', maxsplit=1)
  K=KA[1].split(sep='_', maxsplit=1)

  #Numero de componentes principales
  k_componentes =int(K[1]) 

  #CREACION DE ARCHIVOS CSV CON LAS CARACTERISTICAS
  raiz_histograma_caracteristicas="5.Resultados_Histogramas/"+anexo_archivo
  os.makedirs(raiz_histograma_caracteristicas, exist_ok=True)

  print(raiz_histograma_caracteristicas)
  print(os.path.isdir(raiz_histograma_caracteristicas))
  try:
    print(raiz_histograma_caracteristicas)
    print(os.path.isdir(raiz_histograma_caracteristicas))
  except:
    print("Algo salió mal")
  finally:
    print("El 'intentar excepto' ha terminado")
  
  histograma_caracteristicas=raiz_histograma_caracteristicas+"/histograma_caracteristicas.csv"
  histograma_ruta_redimensionada=raiz_histograma_caracteristicas+"/histograma_ruta_redimensionada.csv"
  histograma_ruta_eigen=raiz_histograma_caracteristicas+"/histograma_ruta_eigen.csv"

  #Guarda la ruta de imagen REDIMENSIONADA
  archivo_ruta_redimensionada= open(histograma_ruta_redimensionada, "w+") 
  #Caracteristicas imagenes eigen a Histogramas
  archivo_histograma= open(histograma_caracteristicas, "w+") 

  #Array para apilar las imagenes
  files = [] #nombre de las imagenes
  path_files = [] #nombre de las carpetas por cantidad de imagenes
  path_database = [] #contiene la ruta de cada imagen

  array_imagenes =[] #matrices que representan cada imagen
  #####---------------------------

  #For que recorre las carpetas(sujetos) del path
  contador = 0
  for aa in os.listdir(path_redimensionadas):
    #For que recorre la imagen de cada carpeta(sujeto)
    for f in os.listdir(path_redimensionadas + '/' + aa):
      contador = contador + 1
      #aa Carpeta del sujeto 
      #f Nombre de la imagen del sujeto
      #Ruta de la imagen
      imgPath = path_redimensionadas + '/' + aa + '/' + f
      files.append(f)
      path_files.append(aa)
      path_database.append(imgPath)

      #Guarda la ruta de imagen REDIMENSIONADA
      archivo_ruta_redimensionada.write(imgPath)
      archivo_ruta_redimensionada.write("\n")

      #Abrir Imagen (Lee la imagen en formtado BGR)
      imgBGR = cv2.imread(imgPath)
      dim = (180, 180)
      #Se redimensiona la imagen de 180 por 180
      resizedBGR = cv2.resize(imgBGR, dim, interpolation = cv2.INTER_AREA) 

      #Se obtiene la información de las imagenes 
      height, width, channels = resizedBGR.shape

      #Convierte la imagen en RGB
      imgRGB = cv2.cvtColor(resizedBGR, cv2.COLOR_BGR2RGB)
      
      #Copiar imagen del formato RGB para luego convertirla en GRAY
      gray_image = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2GRAY)

      #Convertir de Matriz a Vector
      flat_arr = gray_image.ravel()
      #Convertir el vector a un array
      vector = np.array(flat_arr)
      #Se apila a el array
      array_imagenes.append(vector)

  # resul=is_empty(array_imagenes)

  # if (resul==False):
  if (len(array_imagenes)):

    # Se tiene una matriz de todas las imagenes
    matriz_imagenes=np.matrix(array_imagenes)

    #Matriz Promedio (Saca el promedio de todas las imagenes)
    matrix_avg = np.sum(matriz_imagenes,axis=0)/len(matriz_imagenes)

    #Matriz_imagenes - Matriz Promedio
    vec = matriz_imagenes - matrix_avg

    #--------------------------
    #Impresion de la matriz o cara promedio
    # print("Cara Promedio")
    # faceAverage = np.reshape(matrix_avg, (180, 180))
    # plt.figure(figsize=(4, 4))
    # plt.imshow(faceAverage,cmap='Greys')
    # plt.title("Mean Face")
    #--------------------------


    A = np.matrix(vec)
    #print(A)

    #Matriz de Covarianza Reducida
    cov = A*np.transpose(A)


    #--------------------------
    #Se extraen los eigenValores y eigenVectores
    eigenValues, eigenVectors = linalg.eig(cov) 
    idx = eigenValues.argsort()[::-1] 
    eigenValues = eigenValues[idx] 
    eigenVectors = eigenVectors[:,idx] 
    #-------------------------
    #print(eigenVectors[0].shape)
    #print(vec.shape)

    #Numero de componentes principales
    #k_componentes = 5  # Para buscar perro y gato
    #k_componentes = 25  #Gatos
    #k_componentes = 50 #Perros

    k=int(k_componentes)

    #Reconstrucción de imagenes originales por medio de eigenfaces aplicando PCA
    faces_pca = PCA(n_components=k)
    faces_pca.fit(matriz_imagenes)
    components = faces_pca.transform(matriz_imagenes)
    projected = faces_pca.inverse_transform(components)

    #Guarda la ruta de imagen EIGEN
    archivo_ruta_eigen= open(histograma_ruta_eigen, "w+") 

    #Exportacion de eigenfaces a carpeta
    for i,img in enumerate(projected):
      #Guarda imagenes imagenes con cuadros
      os.makedirs(path_resultado+ '/'+ path_files[i], exist_ok=True)
      filename=(path_resultado + '/' + path_files[i] + '/' + files[i]) 
      cv2.imwrite(filename, projected[i].reshape(180,180)) 

      #Guarda la ruta de imagen EIGEN
      archivo_ruta_eigen.write(filename)
      archivo_ruta_eigen.write("\n")

      #Abrir Imagen (Lee la imagen en formtado BGR)
      imgBGR = cv2.imread(filename)

      
      #Convierte la imagen en RGB
      imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
      
      #Copiar imagen del formato RGB para luego convertirla en GRAY
      gray_image = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2GRAY)

      #Se obtiene la información de las imagenes 
      #height, width, channels = gray_image.shape

      
      
      
      #Histograma
      fv_hist = histogram(gray_image)
      i=0
      archivo_histograma.write(filename)
      archivo_histograma.write(",")
      
      for b in fv_hist:
        i=i+1
        archivo_histograma.write("%.20f" % float(b))
        if(i<=(len(fv_hist)-1)):
          archivo_histograma.write(",")
      archivo_histograma.write("\n")
      
      archivo_histograma.close()
      
  else:
    print("Está vacía")



In [ ]:
# Importar Librerias

import dlib, cv2, os
from imutils import face_utils
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import io, color
import cv2 as cv

In [ ]:
def menu_tesis():
  print("----------- Tesis U Catolica -----------")
  print("----------- Seleccione la tecnica a implementar -----------")
  print("1. Eigen")
  print("-----------------------------------")

In [ ]:
def menu_opcion():
  print("----------- Medidas de similaridad -----------")
  print("1. Perros")
  print("2. Buscar Perro")
  print("3. Gatos")
  print("4. Buscar Gato")
  print("5. Gatos QUERY")
  print("6. Perros QUERY")
  print("-----------------------------------")

In [ ]:
def seleccion_menu():
  menu_tesis()
  mopcion = int(input("Seleccione una opcion: "))
  caso_menu(mopcion)

In [ ]:
def seleccion_opcion(mopcion):
  menu_opcion()
  opcion = int(input("Seleccione una opcion: "))
  print(mopcion)
  print(opcion)
  caso_opcion(mopcion,opcion)

In [ ]:
#def caso_opcion(self, mopcion,opcion):
class caso_opcion:
  def __init__(self, mopcion,opcion):

    #*****************
    #      EIGEN
    #*****************

    if mopcion== 1 and opcion == 1: #(1. Eigen, 1. Perros)
      print('1. Eigen, 1. Perros')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Perros' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Perros/ROSTRO_REDIMENSIONADO'#2

      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      #for i in range(1, (contador_imagenes)):  #****CAMBIAR EL VALOR 
      for i in range(530, (contador_imagenes)):
        anexo_archivo='Eigen_Perros'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Perros'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
        
      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))

      
    elif mopcion== 1 and opcion == 2: #(1. Eigen, 2. Buscar Perro)
      print('1. Eigen, 2. Buscar Perro')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Perros_Entrada' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Buscar_perro/ROSTRO_REDIMENSIONADO'#2

      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes)):  #****CAMBIAR EL VALOR 
        anexo_archivo='Eigen_Perros_Entrada'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Perros_Entrada'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
        
      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))

    elif mopcion== 1 and opcion == 3: #(1. Eigen, 3. Gatos)
      print('1. Eigen, 3. Gatos')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Gatos' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Gatos/ROSTRO_REDIMENSIONADO'#2

      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      #for i in range(1, (contador_imagenes)):  #****CAMBIAR EL VALOR 
      for i in range(279, (contador_imagenes)):  #****CAMBIAR EL VALOR 
        anexo_archivo='Eigen_Gatos'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Gatos'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
        print('**'+anexo_archivo)

      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))

    elif mopcion== 1 and opcion == 4: #(1. Eigen, 4. Buscar Gato)
      print('1. Eigen, 4. Buscar Gato')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Gatos_Entrada' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Buscar_gato/ROSTRO_REDIMENSIONADO'#2

      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes)):  #****CAMBIAR EL VALOR 
        anexo_archivo='Eigen_Gatos_Entrada'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Gatos_Entrada'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
        
      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))

    elif mopcion== 1 and opcion == 5: #(1. Eigen, 5. Gatos QUERY)
      print('1. Eigen, 5. Gatos QUERY')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Gatos_QUERY' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Gatos_QUERY/ROSTRO_REDIMENSIONADO'#2 

      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes)):
        anexo_archivo='Eigen_Gatos_QUERY'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Gatos_QUERY'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)

      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))

      #extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
      
    elif mopcion== 1 and opcion == 6: #(1. Eigen, 6. Perros QUERY)
      print('1. Eigen, 6. Perros QUERY')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Perros_QUERY' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Perros_QUERY/ROSTRO_REDIMENSIONADO'#2

      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes)):  #****CAMBIAR EL VALOR 
        anexo_archivo='Eigen_Perros_QUERY'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/Eigen_Perros_QUERY'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
        
      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))
      
    else:
      #seleccion_opcion()
      print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
      print('Error: La opción ingresada de BUSQUEDA no esta dentro de las opciones, por favor vuelva a ingresar la opción')
      print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
      print('')
      print('')
      print('')
      seleccion_opcion()
   
   
    

In [ ]:
def caso_menu(mopcion):
  if mopcion == 1:
    print('Eigen')
    seleccion_opcion(mopcion)
  else:
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    print('Error: La opción ingresada no esta dentro de las opciones, por favor vuelva a ingresar la opción')
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    print('')
    print('')
    seleccion_menu()

seleccion_menu()

----------- Tesis U Catolica -----------
----------- Seleccione la tecnica a implementar -----------
1. Eigen
-----------------------------------
